In [11]:
import rasterio
from rasterio.plot import show
import numpy as np
from pyproj import Proj, transform
import cartopy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import matplotlib.pylab as pl
from ipywidgets import SelectionSlider
from ipyleaflet import *
import ipyleaflet as ipyl
import ipywidgets as ipw
from ipyleaflet import Map, basemaps, basemap_to_tiles, Circle,MarkerCluster, WMSLayer
from math import *
from pvlib import solarposition # see https://pvlib-python.readthedocs.io/en/stable/auto_examples/plot_sunpath_diagrams.html
import pandas as pd
from scipy.stats import binned_statistic
from scipy.interpolate import griddata
from owslib.wcs import WebCoverageService
import PIL.Image as PIL_Image
from matplotlib import cm
import ipywidgets as ipyw
from ipywidgets import HBox,Label,Layout,Box,VBox
from IPython.display import clear_output
import glob
import h5py
from PIL import Image
from shapely.geometry import Point, Polygon
import contextily as cx
from scipy.interpolate import NearestNDInterpolator
import warnings
warnings.filterwarnings('ignore')
from rasterio import logging
import math
from scipy import interpolate
from datetime import datetime
import matplotlib.path as mpltPath
import pickle
log = logging.getLogger()
log.setLevel(logging.FATAL)

# Meta-data measurement site
site_name = 'KNMI-radartoren-Herwijnen'
site_lat = 51.836920486125
site_lon = 5.13806025375357
radar_height = 28 # in meters

# sensor height above roof
sh = 1.5 # for the moment this is a fixed value

#distances_min = [0,0]
#distances_max = [250,5000] # lower value for high resolution, higher value for low resolution

lats_ll = list([])
lats_ur = list([])
lons_ll = list([])
lons_ur = list([])

download_AHN=False
if (download_AHN):
    for ranges in range(0,2):
    #https://service.pdok.nl/rws/ahn3/wcs/v1_0?request=getcapabilities&service=wcs
        print(ranges)
        # specify domain size in meters (square domain centered around sensor)
        if (ranges<1):
            if (ranges==0):
                domain_size = 1000 
                #wcs = WebCoverageService('http://geodata.nationaalgeoregister.nl/ahn3/wcs?service=WCS', version='1.0.0')
                wcs = WebCoverageService('https://service.pdok.nl/rws/ahn3/wcs/v1_0?service=WCS', version='1.0.0')
                cvg = wcs.contents['ahn3_05m_dsm']
        else:
            domain_size = 20000
            #wcs = WebCoverageService('http://geodata.nationaalgeoregister.nl/ahn3/wcs?service=WCS', version='1.0.0')
            wcs = WebCoverageService('https://service.pdok.nl/rws/ahn3/wcs/v1_0?service=wcs', version='1.0.0')
            cvg = wcs.contents['ahn3_5m_dsm']


        # Define projections RDnew (p1) and WGS84 (p2) for coordinate transformations

        # source: https://publicwiki.deltares.nl/display/OET/Python+convert+coordinates
        # copy the SR-ORG:6781 definition in Proj4 format from http://spatialreference.org/ref/sr-org/6781/

        p1 = Proj("+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +towgs84=565.237,50.0087,465.658,-0.406857,0.350733,-1.87035,4.0812 +units=m +no_defs")
        p2 = Proj(proj='latlong',datum='WGS84')


        #https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/intro-to-coordinate-reference-systems-python/

        x,y = transform(p2,p1,site_lon,site_lat)
        bbox = (x-domain_size/2, y-domain_size/2, x+domain_size/2, y+domain_size/2)

        # Download and write TIF file

        if (ranges<1):
            if (ranges==0):
                download = wcs.getCoverage(identifier='ahn3_05m_dsm', bbox=bbox, format='GEOTIFF_FLOAT32',
                                   crs='urn:ogc:def:crs:EPSG::28992', resx=0.5, resy=0.5)
                with open('AHN3_05m_DSM_%s_%d.tif'%(site_name,domain_size), 'wb') as file:
                    file.write(download.read())

                fp = r'AHN3_05m_DSM_%s_%d.tif'%(site_name,domain_size)
        else:
            download = wcs.getCoverage(identifier='ahn3_5m_dsm', bbox=bbox, format='GEOTIFF_FLOAT32',crs='urn:ogc:def:crs:EPSG::28992', resx=5, resy=5)
            with open('AHN3_5m_DSM_%s_%d.tif'%(site_name,domain_size), 'wb') as file:
                file.write(download.read()) 

            fp = r'AHN3_5m_DSM_%s_%d.tif'%(site_name,domain_size)


        img = rasterio.open(fp)
        rasterio.open(fp, driver="GTiff")

        # create lat/lon grid in 2D (use boundbox)
        boundbox = img.bounds
        bb_array = np.array(boundbox)

        lon_ll,lat_ll,z = transform(p1,p2,bb_array[0],bb_array[1],0.0) # ll = lower left corner ; ur = upper right corner
        lon_ur,lat_ur,z = transform(p1,p2,bb_array[2],bb_array[3],0.0)
        lats_ll.append(lat_ll)
        lons_ll.append(lon_ll)
        lats_ur.append(lat_ur)
        lons_ur.append(lon_ur)
        s = img.shape
        print(s)

        # advanced interpolation needed (previous method shows negligible errors along diagonal from ll to ur, but considerable deviations from ul to lr)
        # reason is that lines of constant RDnew (x or y) do not correspond to lines of constant (lat or lon).

        # it would make most sense to convert each point of the grid according to the formal transformation. If this makes the script too slow, then one may choose to transform only every 10th point in x and in y-direction and then to use interpolation in between

        # first test the conventional approach


        lat_1D = np.linspace(lat_ll,lat_ur,s[0]) # 1D axis
        lon_1D = np.linspace(lon_ll,lon_ur,s[1])
        lat_2D,lon_2D = np.meshgrid(lat_1D,lon_1D) # 2D axis 
        # !! this was the old (too much simplified) method. note that this gives the wrong coordinate grid in lat/lon, but the correct shape of the arrays.
        # now replace the values with the correct ones:


        # first make the x-grid
        x_1D = np.linspace(bb_array[1],bb_array[3],s[1])
        y_1D = np.linspace(bb_array[0],bb_array[2],s[0])
        y_2D,x_2D = np.meshgrid(y_1D,x_1D)

        x_1D_lr = np.linspace(bb_array[1],bb_array[3],int(s[1]/100))
        y_1D_lr = np.linspace(bb_array[0],bb_array[2],int(s[0]/100))
        y_2D_lr,x_2D_lr = np.meshgrid(y_1D_lr,x_1D_lr)

        lat_2D_lr = 0*x_2D_lr
        lon_2D_lr = 0*x_2D_lr

        lon_p,lat_p,z = transform(p1,p2,bb_array[0],bb_array[1],0.0)
        print(lat_p,lon_p)

        # double for-loop only at low resolution (_lr)
        for i in range(0,len(x_1D_lr)):
            print(i)
            for j in range(0,len(y_1D_lr)):

                lon_p,lat_p,z = transform(p1,p2,y_2D_lr[i,j],x_2D_lr[i,j],0.0)
                lat_2D_lr[i,j]=lat_p
                lon_2D_lr[i,j]=lon_p

        print(np.amin(lon_2D_lr),np.amax(lon_2D_lr),np.amin(lat_2D_lr),np.amax(lat_2D_lr))

        # convert to high resolution (variable names without "_lr")
        #https://stackoverflow.com/questions/11468367/interp2x-y-z-xi-yi-from-matlab-to-python
        lat_2D = griddata((y_2D_lr.ravel(),x_2D_lr.ravel()),lat_2D_lr.ravel(),(y_2D.ravel(),x_2D.ravel()))
        lon_2D = griddata((y_2D_lr.ravel(),x_2D_lr.ravel()),lon_2D_lr.ravel(),(y_2D.ravel(),x_2D.ravel()))
        lat_2D.resize(img.shape)
        lon_2D.resize(img.shape)


        # calculate distance, elevation and azimuth w.r.t. reference point for every point in the domain
        delta_lat_m = (lat_2D-site_lat)*2*np.pi*6371000./360.
        delta_lon_m = (lon_2D-site_lon)*np.cos(np.radians(site_lat))*2*np.pi*6371000./360.
        distance = np.sqrt(delta_lat_m**2+delta_lon_m**2)


        # convert image data AHN to 2D array
        img_arr = img.read()
        img_arr = img_arr[0,:,:]
        img_arr = np.flip(img_arr,axis=0)##np.flip(np.flip(img_arr.T),axis=0) # transposing and flipping in order to obtain the correct orientation
        img_arr[img_arr<-900]= 'NaN' # surface water
        img_arr[img_arr>900]= 'NaN' # outliers

        
        datasets_layers_ID = 'AHN'#,'elevation','azimuth','distance']
        date_file_creation = '20230709'
        np.save('%s_%s_%s_resolution_%d.npy'%(datasets_layers_ID,site_name,date_file_creation,ranges),[lon_2D,lat_2D,img_arr])
        np.save('%s_%s_%s_lat_lon_ll_ur_%d.npy'%(datasets_layers_ID,site_name,date_file_creation,ranges),[lat_ll,lon_ll,lat_ur,lon_ur])


        # the automatic color scale often is suboptimal in the sense that some high objects in the domain may cause poor contrast for many low objects
        # therefore apply a rescaling in order to optimize color scale

        # - find maximum value of lowest 1% of data (set values below to this maximum value)
        # - [optional] find minimum value of highest 1% of data (set values above to this minimum value)
        # introduce non-linearity in the data such that the color scale is not dominated by the highest buildings which leads to lower contrast for other features
        # then apply a scaling to values between 0 and 255

        img_arr_res = img_arr[:] # note that the rescaling of data is only used for making an image with high contrast in relevant parts of the dataset (do not confuse with the original dataset to be used for calculations)

        img_1D = np.ravel(img_arr_res)
        img_1D = img_1D[~np.isnan(img_1D)]
        percentile_01 = np.percentile(img_1D,1.) 
        percentile_999 = np.percentile(img_1D,99.9) 

        img_arr_res[((~np.isnan(img_arr_res)) & (img_arr_res<percentile_01))]=percentile_01
        img_arr_res[((~np.isnan(img_arr_res)) & (img_arr_res>percentile_999))]=percentile_999

        img_arr_res = 255.*((img_arr_res-percentile_01)/(percentile_999-percentile_01))**(1/4)

        img_res1D = np.ravel(img_arr_res)
        img_res1D = img_res1D[~np.isnan(img_res1D)]

        def create_png_image_layers(dataset,figure_filename):

            fig = plt.figure(figsize=(20, 20))
            ax = plt.axes(projection=ccrs.Mercator()) 
            lon_box_min_map,lon_box_max_map,lat_box_min_map,lat_box_max_map = lon_ll,lon_ur,lat_ll,lat_ur
            ax.set_extent((lon_box_min_map,lon_box_max_map,lat_box_min_map,lat_box_max_map), crs=ccrs.Mercator())

            p=plt.pcolormesh(lon_2D,lat_2D,dataset,transform=ccrs.Mercator())
            plt.draw()
            plt.savefig(figure_filename,bbox_inches='tight',transparent=True, pad_inches=0, dpi=300,format='png')  


            # post-processing of png image in order to remove (=make transparent) black axis elements and white background from figure
            pngimage = PIL_Image.open(figure_filename) # be careful: the function "Image" exists in both the packages PIL and ipywidgets
            pngimage = pngimage.convert("RGBA")
            pixdata = pngimage.load()
            width, height = pngimage.size


            for y in range(height):
                for x in range(width):
                    if pixdata[x, y] == (255, 255, 255, 255):
                        pixdata[x, y] = (255, 255, 255, 0)
                    if pixdata[x, y] == (0, 0, 0, 255):
                        pixdata[x, y] = (255, 255, 255, 0)

            pngimage.save(figure_filename,format="png")
            fig.clf()

        datasets_layers = img_arr_res#,elevation_above_sensorheight,azimuth,distance]

        #save_figure_filenames=[]
        save_figure_filenames='%s_%s_%s_resolution_%d.png'%(datasets_layers_ID,site_name,date_file_creation,ranges)

        
        create_png_image_layers(datasets_layers,save_figure_filenames)
        

In [13]:
def download_AHN_high_lowres(hr_lr,site_lat,site_lon):
    if (hr_lr=='hr'):
        wcs = WebCoverageService('https://service.pdok.nl/rws/ahn3/wcs/v1_0?service=WCS', version='1.0.0')
        cvg = wcs.contents['ahn3_05m_dsm']
    else:
        wcs = WebCoverageService('https://service.pdok.nl/rws/ahn3/wcs/v1_0?service=wcs', version='1.0.0')
        cvg = wcs.contents['ahn3_5m_dsm']
    
    # Define projections RDnew (p1) and WGS84 (p2) for coordinate transformations

    # source: https://publicwiki.deltares.nl/display/OET/Python+convert+coordinates
    # copy the SR-ORG:6781 definition in Proj4 format from http://spatialreference.org/ref/sr-org/6781/

    p1 = Proj("+proj=sterea +lat_0=52.15616055555555 +lon_0=5.38763888888889 +k=0.9999079 +x_0=155000 +y_0=463000 +ellps=bessel +towgs84=565.237,50.0087,465.658,-0.406857,0.350733,-1.87035,4.0812 +units=m +no_defs")
    p2 = Proj(proj='latlong',datum='WGS84')


    #https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/intro-to-coordinate-reference-systems-python/


    # Download and write TIF file

    if (hr_lr=='hr'):
        domain_size=150 # use 3000 only for low resolution
        x,y = transform(p2,p1,site_lon,site_lat)
        bbox = (x-domain_size/2, y-domain_size/2, x+domain_size/2, y+domain_size/2)
        download = wcs.getCoverage(identifier='ahn3_05m_dsm', bbox=bbox, format='GEOTIFF_FLOAT32',
                                   crs='urn:ogc:def:crs:EPSG::28992', resx=0.5, resy=0.5)
        with open('AHN3_05m_DSM_temp.tif', 'wb') as file:
            file.write(download.read())
        fp = r'AHN3_05m_DSM_temp.tif'
    else:
        domain_size=3000 # use 3000 only for low resolution
        x,y = transform(p2,p1,site_lon,site_lat)
        bbox = (x-domain_size/2, y-domain_size/2, x+domain_size/2, y+domain_size/2)
        download = wcs.getCoverage(identifier='ahn3_5m_dsm', bbox=bbox, format='GEOTIFF_FLOAT32',crs='urn:ogc:def:crs:EPSG::28992', resx=5, resy=5)
        with open('AHN3_5m_DSM_temp.tif', 'wb') as file:
            file.write(download.read()) 
        fp = r'AHN3_5m_DSM_temp.tif'


    img = rasterio.open(fp)
    rasterio.open(fp, driver="GTiff")

    # create lat/lon grid in 2D (use boundbox)
    boundbox = img.bounds
    bb_array = np.array(boundbox)

    lon_ll,lat_ll,z = transform(p1,p2,bb_array[0],bb_array[1],0.0) # ll = lower left corner ; ur = upper right corner
    lon_ur,lat_ur,z = transform(p1,p2,bb_array[2],bb_array[3],0.0)
    lats_ll.append(lat_ll)
    lons_ll.append(lon_ll)
    lats_ur.append(lat_ur)
    lons_ur.append(lon_ur)
    s = img.shape

    # advanced interpolation needed (previous method shows negligible errors along diagonal from ll to ur, but considerable deviations from ul to lr)
    # reason is that lines of constant RDnew (x or y) do not correspond to lines of constant (lat or lon).
    # it would make most sense to convert each point of the grid according to the formal transformation. If this makes the script too slow, then one may choose to transform only every 10th point in x and in y-direction and then to use interpolation in between
    # first test the conventional approach

    lat_1D = np.linspace(lat_ll,lat_ur,s[0]) # 1D axis
    lon_1D = np.linspace(lon_ll,lon_ur,s[1])
    lat_2D,lon_2D = np.meshgrid(lat_1D,lon_1D) # 2D axis 
    # !! this was the old (too much simplified) method. note that this gives the wrong coordinate grid in lat/lon, but the correct shape of the arrays.
    # now replace the values with the correct ones:


    # first make the x-grid
    x_1D = np.linspace(bb_array[1],bb_array[3],s[1])
    y_1D = np.linspace(bb_array[0],bb_array[2],s[0])
    y_2D,x_2D = np.meshgrid(y_1D,x_1D)

    x_1D_lr = np.linspace(bb_array[1],bb_array[3],int(s[1]/100))
    y_1D_lr = np.linspace(bb_array[0],bb_array[2],int(s[0]/100))
    y_2D_lr,x_2D_lr = np.meshgrid(y_1D_lr,x_1D_lr)

    lat_2D_lr = 0*x_2D_lr
    lon_2D_lr = 0*x_2D_lr

    lon_p,lat_p,z = transform(p1,p2,bb_array[0],bb_array[1],0.0)

    # double for-loop only at low resolution (_lr)
    for i in range(0,len(x_1D_lr)):
        for j in range(0,len(y_1D_lr)):

            lon_p,lat_p,z = transform(p1,p2,y_2D_lr[i,j],x_2D_lr[i,j],0.0)
            lat_2D_lr[i,j]=lat_p
            lon_2D_lr[i,j]=lon_p


    # convert to high resolution (variable names without "_lr")
    #https://stackoverflow.com/questions/11468367/interp2x-y-z-xi-yi-from-matlab-to-python
    lat_2D = griddata((y_2D_lr.ravel(),x_2D_lr.ravel()),lat_2D_lr.ravel(),(y_2D.ravel(),x_2D.ravel()))
    lon_2D = griddata((y_2D_lr.ravel(),x_2D_lr.ravel()),lon_2D_lr.ravel(),(y_2D.ravel(),x_2D.ravel()))
    lat_2D.resize(img.shape)
    lon_2D.resize(img.shape)

    # convert image data AHN to 2D array
    img_arr = img.read()
    img_arr = img_arr[0,:,:]
    img_arr = np.flip(img_arr,axis=0)##np.flip(np.flip(img_arr.T),axis=0) # transposing and flipping in order to obtain the correct orientation
    img_arr[img_arr<-900]= 'NaN' # surface water
    img_arr[img_arr>900]= 'NaN' # outliers


    #datasets_layers_ID = 'AHN'#,'elevation','azimuth','distance']
    #date_file_creation = '20230709'
    #np.save('%s_%s_%s_resolution_%d.npy'%(datasets_layers_ID,site_name,date_file_creation,ranges),[lon_2D,lat_2D,img_arr])
    #np.save('%s_%s_%s_lat_lon_ll_ur_%d.npy'%(datasets_layers_ID,site_name,date_file_creation,ranges),[lat_ll,lon_ll,lat_ur,lon_ur])
    AHN = [lat_2D,lon_2D,img_arr,[lat_ll,lon_ll,lat_ur,lon_ur]]
    
    return AHN

In [3]:
# model Earth as circle with radius 1 in x-y plane
# implement de 4/3 earth radius model

s = 299792458./1.000325 # speed of light in air
R_earth = 6371.#km

# distance covered in 1 us: (us = microsecond)
d = 1e-6*s

#corresponding angle
theta_1us=d/(R_earth*1000)

t_earth = np.arange(0, 2*math.pi, theta_1us)

# for an elevation angle of 0 degrees, the displaced center is located at x=-1/3, y = 0
#t_earth = np.linspace(0,2*math.pi,1000)
x_earth = R_earth*np.sin(t_earth)
y_earth = R_earth*np.cos(t_earth)

int_h_vs_d = {} # dictionary with interpolators for distance_to_height

alphas=[-0.2,0.3,0.8]
alphas_names=['lower','center','upper']
for ixa in range(0,3): # degrees
    x_4earth_ctr = (4)*R_earth*math.sin(math.radians(alphas[ixa]))
    y_4earth_ctr = 0.001*radar_height+R_earth-(4)*R_earth*math.cos(math.radians(alphas[ixa]))

    t_4earth = np.arange(0,2*math.pi,theta_1us/4.)-(math.pi*alphas[ixa]/180.)

    x_4_earth_circle = x_4earth_ctr+(4)*R_earth*np.sin(t_4earth)
    y_4_earth_circle = y_4earth_ctr+(4)*R_earth*np.cos(t_4earth)

    horizontal_distance = R_earth*np.arcsin(x_4_earth_circle/np.sqrt((x_4_earth_circle)**2+(y_4_earth_circle)**2))
    height_above_ground = np.sqrt((x_4_earth_circle)**2+(y_4_earth_circle)**2)-R_earth
    int_h_vs_d[alphas_names[ixa]] = interpolate.interp1d(horizontal_distance[0:2001], height_above_ground[0:2001])


In [4]:
D=np.load('LatLon_Herwijnen_20230705.npy')
LatArray=D[0]
LonArray=D[1]

In [5]:
#AHN_lowres=np.load('AHN_KNMI-radartoren-Herwijnen_20230709_resolution_1.npy')
#AHN_highres=np.load('AHN_KNMI-radartoren-Herwijnen_20230709_resolution_0.npy')
#fig = plt.figure(figsize=(20, 20))
#ax = plt.axes(projection=ccrs.Mercator()) 
#plt.pcolormesh(AHN_lowres[0],AHN_lowres[1],AHN_lowres[2],transform=ccrs.Mercator())
#plt.show()

In [6]:
#interp_AHN_lowres = NearestNDInterpolator(list(zip(np.ravel(AHN_lowres[0]), np.ravel(AHN_lowres[1]))), np.ravel(AHN_lowres[2]))
#interp_AHN_highres = NearestNDInterpolator(list(zip(np.ravel(AHN_highres[0]), np.ravel(AHN_highres[1]))), np.ravel(AHN_highres[2]))

In [7]:
def distance(s_lat, s_lng, e_lat, e_lng):

   # Approximate radius of earth in km
   R = 6373.0

   s_lat = s_lat*np.pi/180.0
   s_lng = np.deg2rad(s_lng)
   e_lat = np.deg2rad(e_lat)
   e_lng = np.deg2rad(e_lng)

   d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

   return 2 * R * np.arcsin(np.sqrt(d))

In [8]:
def radar_beam_width(int_h_vs_d,d,h):


    #if h is below center height of radar: calculate w_b from delta_h (elevation, range) (elevation-0.25, range)
    #if h is above center height of radar: calculate w_b from delta_h (elevation, range) (elevation+0.25, range)

    f_cntr = int_h_vs_d['center']
    h_cntr = 1000*f_cntr(d)
    f_lower = int_h_vs_d['lower']
    h_lower = 1000*f_lower(d)
    f_upper = int_h_vs_d['upper']
    h_upper = 1000*f_upper(d)

    b_low = (1/np.sqrt(2*math.pi))*np.exp(-0.5*(((h-h_cntr)/(h_cntr-h_lower))**2))

    return b_low


In [9]:
def fraction_blocked_vs_alt(vertical_xs,h):
    
    fract=0*h
    for i in range(0,len(h)):
        fract[i]=100*(len(vertical_xs[np.where(vertical_xs<h[i])])/len(vertical_xs))
    
    return fract

In [10]:
global AHN_lr,AHN_hr,layer_group_polygons,polygon_pixel
m = Map(center=(site_lat,site_lon),zoom=15,layout=Layout(width='100%', height='750px'))
m.add_layer(basemap_to_tiles(basemaps.CartoDB.Positron))
m.add_layer(basemap_to_tiles(basemaps.CartoDB.DarkMatter))
m.add_layer(basemap_to_tiles(basemaps.Esri.WorldImagery))

l1 = {'url': 'https://service.pdok.nl/hwh/luchtfotorgb/wmts/v1_0/2018_ortho25/EPSG:3857/{z}/{x}/{y}.jpeg','opacity':0.5,'name':'Luchtfoto 2018','attribution':'PDOK / Nationaal Georegister'}
l2 = {'url': 'https://service.pdok.nl/hwh/luchtfotorgb/wmts/v1_0/2019_ortho25/EPSG:3857/{z}/{x}/{y}.jpeg','opacity':0.5,'name':'Luchtfoto 2019','attribution':'PDOK / Nationaal Georegister'}
l3 = {'url': 'https://service.pdok.nl/hwh/luchtfotorgb/wmts/v1_0/2020_ortho25/EPSG:3857/{z}/{x}/{y}.jpeg','opacity':0.5,'name':'Luchtfoto 2020','attribution':'PDOK / Nationaal Georegister'}
l4 = {'url': 'https://service.pdok.nl/hwh/luchtfotorgb/wmts/v1_0/Actueel_ortho25/EPSG:3857/{z}/{x}/{y}.jpeg','opacity':0.5,'name':'Luchtfoto Actueel','attribution':'PDOK / Nationaal Georegister'}
m.add_layer(l1)
m.add_layer(l2)
m.add_layer(l3)
m.add_layer(l4)

#AHN_lr = ImageOverlay(url='AHN_KNMI-radartoren-Herwijnen_20230709_resolution_1.png', bounds=((llur_5m[0], llur_5m[1]), (llur_5m[2], llur_5m[3])),name='AHN 5m',opacity=0.5)

#m.add_layer(AHN_lr)

#AHN_hr = ImageOverlay(url='AHN_KNMI-radartoren-Herwijnen_20230709_resolution_0.png', bounds=((llur_05m[0], llur_05m[1]), (llur_05m[2], llur_05m[3])),name='AHN 0.5m',opacity=0.5)

#m.add_layer(AHN_hr)

radar_grid = ImageOverlay(url='latlon_Herwijnen_distant.png',bounds=((51.6,4.5),(52.1,5.5)),name='radar grid distant')
m.add_layer(radar_grid)

radar_grid = ImageOverlay(url='latlon_Herwijnen_close.png',bounds=((51.8,5.0),(51.9,5.2)),name='radar grid close')
m.add_layer(radar_grid)

polygon1 = ipyl.Polygon(locations=[(0, 2), (0, 1), (0, 0)],color="green",fill_color="green") # this is just a polygon to start a layer_group
polygon_pixel = ipyl.Polygon(locations=[(0, 1), (0, 3), (0, 0)],color="green",fill_color="green") # this is just a polygon to start a layer_group
layer_group_polygons=LayerGroup(layers=(polygon1,polygon_pixel),name='Polygons')           



m.add_layer(layer_group_polygons);



m.add_control(FullScreenControl())
control = LayersControl(position='topright')
m.add_control(control)

plot1_mouse = ipyw.widgets.Output()
plot2_mouse = ipyw.widgets.Output()
plot3_mouse = ipyw.widgets.Output()

output_mouse = ipyw.widgets.Output() 
def handle_interaction(**kwargs):
    global layer_group_polygons,polygon_pixel
    #if kwargs.get('type') == 'mousemove':
    if kwargs.get('type') == 'click':
        output_mouse.clear_output(wait=True)
        with output_mouse:
            clear_output(True)
            
            lat_pnt=kwargs.get('coordinates')[0]
            lon_pnt=kwargs.get('coordinates')[1]
            sp=latlon_to_pixel(LatArray,LonArray,lat_pnt ,lon_pnt)
            
            print("selected pixel: (%d, %d) "%(sp[0],sp[1]))
            
            layer_group_polygons.remove_layer(polygon_pixel)
            
            polygon_pixel = ipyl.Polygon(locations=[(LatArray[sp[0],sp[1]], LonArray[sp[0],sp[1]]), (LatArray[sp[0],sp[1]+1], LonArray[sp[0],sp[1]+1]), (LatArray[sp[0]+1,sp[1]+1], LonArray[sp[0]+1,sp[1]+1]),(LatArray[sp[0]+1,sp[1]], LonArray[sp[0]+1,sp[1]])],color="orange") # this is just a polygon to start a layer_group

            layer_group_polygons.add_layer(polygon_pixel)
            
            
        plot1_mouse.clear_output(wait=True)
        with plot1_mouse:
            
            clear_output(True)
            #fig, ax = plt.subplots(nrows=1,ncols=2,subplot_kw={'projection': ccrs.Mercator()},figsize=(30,10))
            fig, ax = plt.subplots(nrows=1,ncols=1,subplot_kw={'projection': ccrs.Mercator()},figsize=(10,7))
            
            #'url': 'https://{s}.basemaps.cartocdn.com/{variant}/{z}/{x}/{y}{r}.png'
            #tiler = cimgt.Stamen('terrain-background')
            #tiler = cimgt.Stamen('terrain')
            
            #tiler=cimgt.GoogleTiles(desired_tile_form='Positron', style='street', url='https://{s}.basemaps.cartocdn.com/{variant}/{z}/{x}/{y}{r}.png')
            #tiler=cimgt.GoogleTiles(desired_tile_form='RGB',url='https://{s}.basemaps.cartocdn.com/{variant}/{z}/{x}/{y}{r}.png')
            tiler=cimgt.GoogleTiles(desired_tile_form='RGB', style='satellite', url='https://mts0.google.com/vt/lyrs={style}@177000000&hl=en&src=api&x={x}&y={y}&z={z}&s=G')
            zoom=15
            ax.add_image(tiler,zoom)

              
            #https://dlab.berkeley.edu/news/adding-basemaps-python-contextily
            #print(cx.providers.CartoDB.keys())
            #crs='EPSG:3857'
            #cx.add_basemap(ax,zoom,  source=cx.providers.CartoDB.Positron)
            
            # calculate pixel center, pixel dlat, dlon
            pixel_center_lat = (LatArray[sp[0],sp[1]]+LatArray[sp[0]+1,sp[1]+1])/2.
            pixel_delta_lat = abs(LatArray[sp[0],sp[1]]-LatArray[sp[0]+1,sp[1]+1])
            pixel_center_lon = (LonArray[sp[0],sp[1]]+LonArray[sp[0]+1,sp[1]+1])/2. 
            pixel_delta_lon = abs(LonArray[sp[0],sp[1]]-LonArray[sp[0]+1,sp[1]+1])
            
            # distance to radar
            dist=distance(site_lat,site_lon,pixel_center_lat,pixel_center_lon)
            print("dist. of pixel center to radar: %.2f km"%dist)
            
            f=int_h_vs_d['center']
            z_beam_center=f(dist)
            f=int_h_vs_d['lower']
            z_beam_lower=f(dist)
            f=int_h_vs_d['upper']
            z_beam_upper=f(dist)
            delta_max = np.amax([pixel_delta_lat,pixel_delta_lon])
            
#             pixel_in_lr=False
#             # decide if pixel is within bounding box AHN 5m download (AHN_KNMI-radartoren-Herwijnen_20230705b_lat_lon_ll_ur_0.npy)
#             if ((pixel_center_lat>llur_5m[0]) & (pixel_center_lon>llur_5m[1]) & (pixel_center_lat<llur_5m[2]) & (pixel_center_lon<llur_5m[3])):
#                 #print("pixel inside of domain AHN")
#                 pixel_in_lr=True
#             #else:
#                 #print("pixel outside of domain AHN")

#             pixel_in_hr=False
#             # decide if pixel is within bounding box AHN 0.5m download (AHN_KNMI-radartoren-Herwijnen_20230705b_lat_lon_ll_ur_0.npy)
#             if ((pixel_center_lat>llur_05m[0]) & (pixel_center_lon>llur_05m[1]) & (pixel_center_lat<llur_05m[2]) & (pixel_center_lon<llur_05m[3])):
#                 #print("pixel inside of domain AHN")
#                 pixel_in_hr=True
            #else:
                
                #print("pixel outside of domain AHN")


                
                
            ll=[LonArray[sp[0],sp[1]],LatArray[sp[0],sp[1]]]
            lr=[LonArray[sp[0]+1,sp[1]],LatArray[sp[0]+1,sp[1]]]
            ul=[LonArray[sp[0],sp[1]+1],LatArray[sp[0],sp[1]+1]]
            ur=[LonArray[sp[0]+1,sp[1]+1],LatArray[sp[0]+1,sp[1]+1]]
            ax.plot([LonArray[sp[0],sp[1]],LonArray[sp[0],sp[1]+1],LonArray[sp[0]+1,sp[1]+1],LonArray[sp[0]+1,sp[1]],LonArray[sp[0],sp[1]]],[LatArray[sp[0],sp[1]],LatArray[sp[0],sp[1]+1],LatArray[sp[0]+1,sp[1]+1],LatArray[sp[0]+1,sp[1]],LatArray[sp[0],sp[1]]],transform=ccrs.PlateCarree(),color='white')         
            ax.scatter([ll[0],lr[0]],[ll[1],lr[1]],s=40,color='blue',transform=ccrs.PlateCarree())
            ax.scatter([ul[0],ur[0]],[ul[1],ur[1]],s=40,color='red',transform=ccrs.PlateCarree())


            local_grid_lat = np.zeros((401,301))
            local_grid_lon = np.zeros((401,301))
            
            local_grid_AHN_hr = np.zeros((401,301))
            local_grid_AHN_lr = np.zeros((401,301))
            
            AHN_hr = download_AHN_high_lowres('hr',lat_pnt, lon_pnt)
            AHN_lon_hr = AHN_hr[1]
            AHN_lat_hr = AHN_hr[0]
            AHN_z_hr = AHN_hr[2]
            AHN_lr = download_AHN_high_lowres('lr',lat_pnt, lon_pnt)
            AHN_lon_lr = AHN_lr[1]
            AHN_lat_lr = AHN_lr[0]
            AHN_z_lr = AHN_lr[2]
            
            
            interp_AHN_temp_hr = NearestNDInterpolator(list(zip(np.ravel(AHN_lon_hr), np.ravel(AHN_lat_hr))), np.ravel(AHN_z_hr))
            interp_AHN_temp_lr = NearestNDInterpolator(list(zip(np.ravel(AHN_lon_lr), np.ravel(AHN_lat_lr))), np.ravel(AHN_z_lr))



            

            for ix_theta in range(0,401):
                support_point_lon_lower = ll[0]+ix_theta*(lr[0]-ll[0])/400.
                support_point_lat_lower = ll[1]+ix_theta*(lr[1]-ll[1])/400.
                support_point_lon_upper = ul[0]+ix_theta*(ur[0]-ul[0])/400.
                support_point_lat_upper = ul[1]+ix_theta*(ur[1]-ul[1])/400.                

                #ax.plot([support_point_lon_lower,support_point_lon_upper],[support_point_lat_lower,support_point_lat_upper],color='orange',transform=ccrs.PlateCarree())    
                #ax.scatter([support_point_lon_lower,support_point_lon_upper],[support_point_lat_lower,support_point_lat_upper],s=10,color='orange',transform=ccrs.PlateCarree())    
                for ix_radial in range(0,301):
                    lons_radial=support_point_lon_lower+ix_radial*(support_point_lon_upper-support_point_lon_lower)/300.
                    lats_radial=support_point_lat_lower+ix_radial*(support_point_lat_upper-support_point_lat_lower)/300.
                    #ax.scatter(lons_radial,lats_radial,s=10,color='yellow',transform=ccrs.PlateCarree())
                
#                     if (pixel_in_lr):
#                         local_grid_AHN_lowres[ix_theta,ix_radial] = interp_AHN_lowres(lons_radial, lats_radial)
                    local_grid_AHN_hr[ix_theta,ix_radial]=interp_AHN_temp_hr(lons_radial, lats_radial)
                    local_grid_AHN_lr[ix_theta,ix_radial]=interp_AHN_temp_lr(lons_radial, lats_radial)
                    local_grid_lat[ix_theta,ix_radial]=lats_radial
                    local_grid_lon[ix_theta,ix_radial]=lons_radial
#                     if (pixel_in_hr):
#                         local_grid_AHN_highres[ix_theta,ix_radial] = interp_AHN_highres(lons_radial, lats_radial)
                    


                        
#             if (pixel_in_lr):
#                 if (pixel_in_hr!=True):
#                     plt.pcolormesh(local_grid_lon,local_grid_lat,local_grid_AHN_lowres,transform=ccrs.PlateCarree())
#             if (pixel_in_hr):
#                     plt.pcolormesh(local_grid_lon,local_grid_lat,local_grid_AHN_highres,transform=ccrs.PlateCarree())
            
            plt.pcolormesh(local_grid_lon,local_grid_lat,local_grid_AHN_hr,transform=ccrs.PlateCarree())
            
            ax.patch.set_alpha(0)
            ax.set_extent((pixel_center_lon-2*delta_max,pixel_center_lon+2*delta_max,pixel_center_lat-1.5*delta_max,pixel_center_lat+1.5*delta_max))

            plt.show()

        plot2_mouse.clear_output(wait=True)
        with plot2_mouse:
            clear_output(True)
            
#             if (pixel_in_lr):
#                 fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,7))
#                 plt.plot(np.nanmax(local_grid_AHN_lowres,axis=1),label='AHN 5m')
#             if (pixel_in_hr):
#                 plt.plot(np.nanmax(local_grid_AHN_highres,axis=1),label='AHN 0.5m')
#             if (pixel_in_lr):
#                 plt.xlabel('1 degree in azimuthal direction')
#                 plt.ylabel('AHN')
#                 plt.title('Cross section for selected pixel')
#                 plt.legend()
#                 plt.grid()
#                 plt.show()

            fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,7))
            
            xs_hr = np.nanmax(local_grid_AHN_hr,axis=1)
            xs_lr = np.nanmax(local_grid_AHN_lr,axis=1)

            plt.plot(xs_hr,label='surface height (SH) AHN 0.5m')
            plt.plot(xs_lr,label='surface height (SH) AHN 5m')
            
            nonnanminxs_hr = np.nanmin(xs_hr)
            xs_hr[np.where(xs_hr!=xs_hr)]=nonnanminxs_hr
            nonnanminpixel_hr=np.nanmin(local_grid_AHN_hr)
            local_grid_AHN_hr[np.where(local_grid_AHN_hr!=local_grid_AHN_hr)]=nonnanminpixel_hr
            mean_pixel_height_hr=np.mean(local_grid_AHN_hr)
            mean_xs_height_hr=np.mean(xs_hr)

            nonnanminxs_lr = np.nanmin(xs_lr)
            xs_lr[np.where(xs_lr!=xs_lr)]=nonnanminxs_lr
            nonnanminpixel_lr=np.nanmin(local_grid_AHN_lr)
            local_grid_AHN_lr[np.where(local_grid_AHN_lr!=local_grid_AHN_lr)]=nonnanminpixel_lr
            mean_pixel_height_lr=np.mean(local_grid_AHN_lr)
            mean_xs_height_lr=np.mean(xs_lr)
 
            
            plt.plot([0,400],[mean_pixel_height_hr,mean_pixel_height_hr],color='black',linestyle='dashed',label='pixel averaged (AHN 0.5m)')
            plt.plot([0,400],[mean_xs_height_hr,mean_xs_height_hr],color='black',linestyle='dashdot',label='average cross section (AHN 0.5m)')
            plt.plot([0,400],[mean_pixel_height_lr,mean_pixel_height_lr],color='gray',linestyle='dashed',label='pixel averaged (AHN 5m)')
            plt.plot([0,400],[mean_xs_height_lr,mean_xs_height_lr],color='gray',linestyle='dashdot',label='average cross section (AHN 5m)')
            plt.plot([0,400],[1000*z_beam_lower,1000*z_beam_lower],color='purple',label='radar beam lower (%.0f m.)'%(1000*z_beam_lower))
            plt.plot([0,400],[1000*z_beam_center,1000*z_beam_center],color='purple',label='radar beam center (%.0f m.)'%(1000*z_beam_center))
            plt.plot([0,400],[1000*z_beam_upper,1000*z_beam_upper],color='purple',label='radar beam upper (%.0f m.)'%(1000*z_beam_upper))


            plt.xlabel('400x0.0025 degree in azimuthal direction')
            plt.ylabel('height (m) above sea level')
            plt.title('Cross section for selected radar pixel (Herwijnen)')
            plt.ylim([-10,1.2*np.nanmax(xs_hr)])
            plt.legend()
            plt.grid()
            plt.show()

        plot3_mouse.clear_output(wait=True)
        with plot3_mouse:
            clear_output(True)

            fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(10,7))
            h = np.linspace(0,8000,8001)
            beam=radar_beam_width(int_h_vs_d,dist,h)

            f_z = fraction_blocked_vs_alt(xs_hr,h)
            plt.plot(beam/np.max(beam),h,label='radar beam width')
            plt.plot(f_z/np.max(f_z),h,label='fraction of beam not blocked')
            plt.xlabel('normalized values')
            plt.ylabel('height above ground')
            plt.ylim([-10,3*h[np.where(beam==np.max(beam))]])
            plt.legend()
            plt.title('Radar beam width & fraction above horizon')
            plt.grid()
            plt.show()

m.on_interaction(handle_interaction)

def latlon_to_pixel(LatArray,LonArray,lat_pnt ,lon_pnt):
    selected_point=Point(lat_pnt, lon_pnt)
    
    s = np.shape(LonArray)
    selected_pixel=[-9,-9]
    search = True
    for ix in range(0,s[0]-1):
        if (search):
            for iy in range(0,s[1]-1):
                if (search):
                    pixel_polygon = []
                    pixel_polygon = Polygon([(LatArray[ix,iy],LonArray[ix,iy]),(LatArray[ix+1,iy],LonArray[ix+1,iy]),(LatArray[ix+1,iy+1],LonArray[ix+1,iy+1]),(LatArray[ix,iy+1],LonArray[ix,iy+1]),(LatArray[ix,iy],LonArray[ix,iy])])

                    if (selected_point.within(pixel_polygon)):
                        selected_pixel=[ix,iy]
                        #print(pixel_polygon)
                        search = False
                        
                        #polygon = Polygon(locations=pixel_polygon,
                        #color="green",
                        #fill_color="green"
                        #)
                        #print(m)
                        
                        #m.add_layer(polygon);
    print('%.3f %.3f'%(lat_pnt,lon_pnt))
    return selected_pixel


#def select_polygon(radar_grid_tupples,lat_pt,lon_pt):
    
    
#    return(index)


display(m,output_mouse,Box([plot1_mouse,plot2_mouse,plot3_mouse]))

Map(center=[51.836920486125, 5.13806025375357], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

Output()

Box(children=(Output(), Output(), Output()))